In [20]:
%load_ext autoreload
%autoreload 2
from module_test.raw_code.optionlib_2.vol.implied_vol import (
    bsm_vol_est_brute_force,
    bsm_vol_est_minimization,
    vector_vol_estimation
)
from module_test.raw_code.optionlib_2.assets.forward import (
    EquityForward, 
    time_distance_helper,
    vectorized_market_forward_calc
)
from datetime import datetime
from dbase.DataAPI.ThetaData import retrieve_chain_bulk
from trade.helpers.helper import change_to_last_busday, retrieve_timeseries
import os
os.environ['PROXY_URL'] = ''
def get_spot(tick, date):
    return retrieve_timeseries(tick, date, date)['close'][0]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import pandas as pd
pd.options.plotting.backend = "plotly"

In [9]:
test_start, test_valuation_date = '2025-07-16', '2025-07-16'
mkt_forward = EquityForward(
    start_date=test_start,
    end_date=datetime(2025, 12, 19),
    ticker='AAPL',
    valuation_date=test_valuation_date,
    risk_free_rate=None,
    dividend_type='discrete',
    dividend=None,  # Market dividend will be set later

)
mkt_forward.get_forward_price(), mkt_forward.risk_free_rate

(213.46615757988374, 0.0423199987411499)

In [11]:
rates = mkt_forward.risk_free_rate
bsm_vol_est_minimization(
    F=mkt_forward.get_forward_price(),  # Forward price
    K=220,     # Strike price
    T=time_distance_helper('2025-12-19', test_valuation_date),     # Time to maturity in years
    r=mkt_forward.risk_free_rate,   # Risk-free rate
    market_price=11.85,  # Market price of the option
    option_type='c'     # Option type: 'c' for call
)


0.2677398648854257

In [12]:
bsm_vol_est_brute_force(
    F=mkt_forward.get_forward_price(),  # Forward price
    K=220,     # Strike price
    T=time_distance_helper('2025-12-19', test_start),     # Time to maturity in years
    r=mkt_forward.risk_free_rate,   # Risk-free rate
    market_price=11.85,  # Market price of the option
    option_type='c'     # Option type: 'c' for call
)

0.2677033175829395

#### Test Vol Surface Fit

In [18]:
aapl_chain=retrieve_chain_bulk(
    'AAPL',
    0,
    change_to_last_busday(test_valuation_date),
    change_to_last_busday(test_valuation_date),
    '16:00'

)
S = get_spot('AAPL', (test_valuation_date))

In [19]:
aapl_chain = aapl_chain[aapl_chain['Expiration'] >= test_valuation_date]
valuation_dates = [test_valuation_date] * len(aapl_chain)
end_dates = aapl_chain['Expiration'].tolist()
r = [rates] * len(aapl_chain)
s = [S] * len(aapl_chain)
tickers = ['AAPL'] * len(aapl_chain)
F = vectorized_market_forward_calc(
    ticks=tickers,
    S=s,
    valuation_dates=valuation_dates,
    end_dates=end_dates,
    r=r,
    div_type='discrete'
)
F

array([211.06290119, 211.23415549, 211.06290119, ..., 210.72080899,
       217.71608065, 217.71608065])

In [21]:
bsm_vol_est_brute_force?

Signature:
bsm_vol_est_brute_force(
    F: float,
    K: float,
    T: float,
    r: float,
    market_price: float,
    option_type: str = 'c',
)
Docstring:
Brute force method to estimate implied volatility by minimizing the difference
between the market price and the Black-Scholes price.
Parameters:
- F: Forward price
- K: Strike price
- T: Time to maturity
- r: Risk-free rate
- market_price: Market price of the option
- option_type: 'c' for call, 'p' for put
Returns:
- Estimated volatility
File:      ~/cloned_repos/QuantTools/module_test/raw_code/optionlib_2/vol/implied_vol.py
Type:      function

In [26]:
params = list(zip(
    F, 
    aapl_chain['Strike'], 
    [time_distance_helper(end_date, test_valuation_date) for end_date in aapl_chain['Expiration']], 
    r, 
    aapl_chain['Midpoint'], 
    aapl_chain['Right'].str.lower()
))


In [27]:
full_vol = vector_vol_estimation(bsm_vol_est_brute_force, params)
full_vol

[0.29094924873121825,
 0.2747020925523138,
 0.27845143628590713,
 0.28082602065051626,
 0.27020288007200177,
 0.26982794569864244,
 0.26320410510262754,
 0.27107772694317356,
 0.2627041926048151,
 0.26620358008950223,
 0.2628291707292682,
 0.2707027925698142,
 0.2752020050501262,
 0.27570191754793866,
 0.2845753643841096,
 0.28157588939723494,
 0.23483407085177127,
 0.316444786119653,
 0.24070804270106752,
 0.31319535488387207,
 0.2683282082052051,
 0.27407720193004825,
 0.2687031425785644,
 0.27095274881872045,
 0.276076851921298,
 0.2870749268731718,
 0.2869499487487187,
 0.276076851921298,
 0.24345756143903596,
 0.2745771144278607,
 0.29657326433160824,
 0.22383599589989747,
 0.3115706392659816,
 0.32756783919597987,
 0.2961983299582489,
 0.3080712517812945,
 0.2969481987049676,
 0.31382024550613763,
 0.27095274881872045,
 0.2770766769169229,
 0.2793262831570789,
 0.2733273331833296,
 0.28857466436660917,
 0.2958233955848896,
 0.27570191754793866,
 0.26807825195629886,
 0.2700779019

In [35]:
aapl_chain['ImpliedVol'] = full_vol
aapl_chain.Expiration.unique()
aapl_chain[(aapl_chain['Expiration'] == '2026-12-18') & (aapl_chain['Right'] == 'C')].sort_values('Strike').tail(60).plot(y = 'ImpliedVol', x='Strike', kind='line', title='AAPL Call Options Implied Volatility')